In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Flatten
from keras import callbacks
from keras import backend, optimizers

/home/bgalvao/miniconda3/envs/tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# load data
import pickle
with open('dataset.npys', 'rb') as handle:
    train, train_labels, test, test_labels = pickle.load(handle)
train.shape, train_labels.shape, test.shape

((308, 50, 50, 3), (308, 2), (166, 50, 50, 3))

The neural network is defined in the following code block. The `strides` for the pooling layers are defaulted to the `pool_size`, and thus explicitly set to `(1,1)`.

In [3]:
# define model
backend.clear_session()  # resets graph, we don't need multiples of the same
model = Sequential()
model.add(Conv2D(32, kernel_size=(9,9), input_shape=(50, 50, 3)))
model.add(MaxPooling2D(pool_size=(3,3), strides=(1,1)))
model.add(Activation('relu'))
model.add(Conv2D(64, kernel_size=(4,4)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

Here we define the optimizer. Here is a [view on gradient descent optimization algorithms](http://ruder.io/optimizing-gradient-descent/index.html). The suggested optimizer by Keras, RMSProp, hits a local optimum (though its default learning rate may have been to high) really early in the training. Thus I switched to other optimizers.

What is more, I can only get the accuracy metric from `keras`. I also want to take a look at other metrics such as precision and recall - mainly inspired by this [post and the accuracy paradox](https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/). Let us recall (pun definitely intended) what these metrics are. For ${TP}$, ${TN}$, ${FP}$ and ${FN}$ standing for some combination between {**T**rue, **F**alse} and {**P**ositives, **N**egatives}.


- _**Accuracy**_ is the number of correct predictions divided by the total number of predictions $\frac{TN + TP}{TN + TP + FP + FN}$
- _**Precision**_ is the number of positive predictions divided by the total number of positive class values predicted, thus $\frac{TP}{TP+FP}$
- _**Recall**_ is the number of True Positives divided by the number of True Positives and the number of False Negatives, hence $\frac{TP}{TP+FN}$
- _**F1 Score**_ is meant to convey the balance between precision and recall. It's form is $2*\frac{{precision}*{recall}}{{precision}+{recall}}$

While the accuracy could be sufficient given that the classes are balanced (I explicitly split the classes 50/50), this cannot always be the case, and it is important to know how to compute other measures using `keras`, which is something I plan on doing later.

In [4]:
# define optimizer
optmzr = optimizers.SGD(lr=0.005)
#optmzr = optimizers.Adam(lr=0.005)

# define metrics
def precision(y_true, y_pred):
    pass

def recall(y_true, y_pred):
    pass

def f1(y_true, y_pred):
    pass

# For a binary classification problem
model.compile(optimizer=optmzr,
              loss='binary_crossentropy',
              metrics=['accuracy'])

The `tensorboard_callback` allows to log the results to the specified directory `'./log_keras'`. So later, this log can be visualized using the command `tensorboard --logdir log_keras`. Personally, it could be interesting to also add this [precision-recall curve](https://medium.com/@akionakas/precision-recall-curve-with-keras-cd92647685e1).

In [5]:
# hyper and meta params
iterations = 20
batch = 32
tensorboard_callback = callbacks.TensorBoard(
    log_dir='./log_keras',
    histogram_freq=0,
    write_graph=True,
    write_images=True
)

model.fit(
    train,
    train_labels,
    epochs=iterations,
    batch_size=batch,
    callbacks=[tensorboard_callback]
)

Epoch 1/20
308/308 [==============================] - 8s 24ms/step - loss: 0.7058 - acc: 0.4740
Epoch 2/20
308/308 [==============================] - 8s 27ms/step - loss: 0.6511 - acc: 0.5487
Epoch 3/20
308/308 [==============================] - 8s 27ms/step - loss: 0.6405 - acc: 0.6299
Epoch 4/20
308/308 [==============================] - 8s 27ms/step - loss: 0.6441 - acc: 0.6494
Epoch 5/20
308/308 [==============================] - 9s 28ms/step - loss: 0.6497 - acc: 0.6234
Epoch 6/20
308/308 [==============================] - 10s 32ms/step - loss: 0.6670 - acc: 0.5942
Epoch 7/20
308/308 [==============================] - 9s 30ms/step - loss: 0.6348 - acc: 0.6753
Epoch 8/20
308/308 [==============================] - 9s 30ms/step - loss: 0.6272 - acc: 0.6818
Epoch 9/20
308/308 [==============================] - 10s 32ms/step - loss: 0.5752 - acc: 0.7338
Epoch 10/20
308/308 [==============================] - 9s 30ms/step - loss: 0.6328 - acc: 0.6494
Epoch 11/20
308/308 [================

In [6]:
score = model.evaluate(test, test_labels, batch_size=12)
print('On the test set')
print('loss ::', round(score[0], 2))
print('accu ::', round(score[1], 2))

166/166 [==============================] - 2s 11ms/step
On the test set
loss :: 0.51
accu :: 0.77


This run got 77% accuracy on the test set! Not bad, but definitely improvable. :) Keep in mind however, this was trained on a cpu and a very small subset of the data, so it's not much to celebrate haha.

Anyways, if you want to check out details of this run, run the command `tensorboard --logdir log_keras` and open `localhost:6006` on the browser.